In [1]:
import tangram as tg
import anndata
import torch

# Check if GPU is available for faster training, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [2]:
# Load single-cell and spatial data

sc_path = r"..\data\processed\ad_sc_pca.h5ad"
sp_path = r"..\data\processed\ad_sp_pca.h5ad"



print("Loading single-cell data...")
sc_data = anndata.read_h5ad(sc_path)
print(f"Single-cell data loaded. Shape: {sc_data.shape}")

print("Loading spatial data...")
sp_data = anndata.read_h5ad(sp_path)
print(f"Spatial data loaded. Shape: {sp_data.shape}")

Loading single-cell data...
Single-cell data loaded. Shape: (3500, 50)
Loading spatial data...
Spatial data loaded. Shape: (3500, 50)


In [3]:

# Preprocessing
print("Finding common genes between single-cell and spatial data...")
common_genes = sc_data.var_names.intersection(sp_data.var_names)
print(f"Number of common genes: {len(common_genes)}")

print("Filtering datasets to keep only common genes...")
sc_data = sc_data[:, common_genes]
sp_data = sp_data[:, common_genes]
print(f"Filtered single-cell data shape: {sc_data.shape}")
print(f"Filtered spatial data shape: {sp_data.shape}")

# Normalize single-cell data
print("Normalizing single-cell and spatial data...")
tg.pp_adatas(sc_data, sp_data, genes=common_genes)
print("Normalization complete.")
print(f"Training genes used: {sc_data.uns['training_genes']}")

INFO:root:50 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:50 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


Finding common genes between single-cell and spatial data...
Number of common genes: 50
Filtering datasets to keep only common genes...
Filtered single-cell data shape: (3500, 50)
Filtered spatial data shape: (3500, 50)
Normalizing single-cell and spatial data...
Normalization complete.
Training genes used: ['0', '20', '46', '41', '26', '35', '40', '33', '12', '6', '7', '34', '45', '31', '14', '32', '2', '18', '19', '23', '22', '13', '37', '47', '38', '29', '9', '49', '42', '25', '44', '16', '5', '15', '1', '21', '3', '30', '48', '43', '28', '4', '36', '39', '17', '24', '11', '27', '10', '8']


In [4]:
# Train Tangram model
print("Training Tangram model... (This may take some time)")
tangram_model = tg.map_cells_to_space(
    sc_data,
    sp_data,
    device=device,
    num_epochs=500,
    learning_rate=1e-3
)
print("Model training completed.")

# Save the mapped data to a file
tangram_model.write('../data/processed/tangram_model.h5ad')

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 50 genes and rna_count_based density_prior in cells mode...


Training Tangram model... (This may take some time)


INFO:root:Printing scores every 100 epochs.


Score: 0.008
Score: 0.065
Score: 0.126
Score: 0.192
Score: 0.260


INFO:root:Saving results..


Model training completed.


In [ ]:
import scanpy as sc

# Load the saved mapped data
tangram_model = sc.read_h5ad('tangram_model.h5ad')


In [7]:
tg.project_genes(adata_map=tangram_model, adata_sc=sc_data)


AnnData object with n_obs × n_vars = 3500 × 50
    obs: 'uniform_density', 'rna_count_based_density'
    var: 'n_cells', 'sparsity', 'is_training'
    uns: 'training_genes', 'overlap_genes'

In [ ]:
tg.project_cell_annotations(tangram_model, s_data, annotation="cell_subclass")
annotation_list = list(pd.unique(sc_data.obs['cell_subclass']))
tg.plot_cell_annotation_sc(sp_data, annotation_list,perc=0.02)

AttributeError: 'function' object has no attribute 'obs'

In [6]:
# Save the trained model
model_path = "../data/processed/tangram_trained_model.pth"
print(f"Saving trained model to {model_path}...")
sp_data.write_h5ad(model_path)

print("Model saved successfully.")

Saving trained model to ../data/processed/tangram_trained_model.pth...
Model saved successfully.


# 📌 Visualization 

In [9]:
import tangram as tg
import anndata
import torch
import matplotlib.pyplot as plt

# Load the trained spatial data

model_path = "..\data\processed\tangram_trained_model.pth"



In [11]:


# # Paths to saved files
# trained_sp_data_path = sc_data.h5ad
# model_path = "..\data\processed\tangram_trained_model.pth"

# # Load the spatial data
# print(f"Loading trained spatial data from {trained_sp_data_path}...")
# sp_data = anndata.read_h5ad(trained_sp_data_path)
# print("Trained spatial data loaded successfully.")

# Load the trained Tangram model
print(f"Loading trained model from {model_path}...")
tangram_model = tg.map_cells_to_space
tangram_model.load_state_dict(torch.load(model_path))
print("Trained model loaded successfully.")




Loading trained model from ..\data\processed	angram_trained_model.pth...


AttributeError: 'function' object has no attribute 'load_state_dict'

In [12]:
# Visualizing spatial cell-type mapping
print("Visualizing spatial cell-type mapping...")
tg.plot_cell_type_maps(sp_data, figsize=(15, 15))
plt.show()

Visualizing spatial cell-type mapping...


AttributeError: module 'tangram' has no attribute 'plot_cell_type_maps'

# New